In [0]:
current_user_ = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
new_path = f"tmp_directory/{current_user_}"

In [0]:
new_path

In [0]:
# remove inital tmp folder if exists
dbutils.fs.rm(f"dbfs:/{new_path}",True)

In [0]:
dbutils.fs.mkdirs(f"dbfs:/{new_path}")

### Check if it really fails to append, on very simple example

In [0]:
# first run is working, in the same way as epoch 0 did not throw an error, because the file has been just created
with open(f"/dbfs/{new_path}/results.csv", "a") as f:
  f.write("Now the file has more content! \n")

In [0]:
# second run (real append) is not working
with open(f"/dbfs/{new_path}/results.csv", "a") as f:
  f.write("Now the file has more content! \n")

In [0]:
# here we have only one line, which has been written to file, while creating it, second one has thrown an error and it does not appear here
with open(f"/dbfs/{new_path}/results.csv", "r") as f:
  print(f.read())

Remark: saving on mounted point, for example Azure Blob Storage, also did not help


```
with open("/dbfs/mnt/{name_of_your_mount_point}/{path_to_file}/results.csv", "a") as f:
  f.write("Now the file has more content! \n")
  
with open("/dbfs/mnt/{name_of_your_mount_point}/{path_to_file}/results.csv", "a") as f:
  f.write("Now the file has more content! \n")
```
  
```
OSError: [Errno 95] Operation not supported
```

### Solution - append in local filesystem on driver node and then move where one wants

In [0]:
with open("/databricks/driver/results1.csv", "a") as f:
  f.write("Now the file has more content! \n")

In [0]:
with open("/databricks/driver/results1.csv", "a") as f:
  f.write("Now the file has more content! \n")

In [0]:
with open("/databricks/driver/results1.csv", "a") as f:
  f.write("Now the file has more content! \n")

In [0]:
%fs ls file:/databricks/driver

In [0]:
# copy to dbfs
dbutils.fs.mv("file:/databricks/driver/results1.csv", f"dbfs:/{new_path}/results1.csv")

In [0]:
# we see that append happened
with open(f"/dbfs/{new_path}/results1.csv", "r") as f:
  print(f.read())

In [0]:
# we remove this file
dbutils.fs.rm("/databricks/driver/results1.csv")

In [0]:
%fs ls file:/databricks/driver